In [43]:
import pandas as pd
import numpy as np

def short_rate_tree(n,init_rate,u,d):
    tree = np.zeros((n+1,n+1))
    for i in range(n+1):
        for j in range(i+1):
            tree[i][j]=r00*u**j*d**(i-j)
    rate_df = pd.DataFrame(data=tree, columns=np.arange(tree.shape[0]))
    return rate_df

def swap_price_lattice(short_rate,notional=1000000.0,fixed_rate = 0.045):
    n = short_rate.shape[0]-1
    rate_tree= short_rate.to_numpy()
    
    payoff = np.zeros((n+1,n+1))
    for i in range(n+1):
        payoff[n,i]=(rate_tree[n,i]-fixed_rate)/(1+rate_tree[n,i])
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            payoff[i,j]= (rate_tree[i,j]-fixed_rate+q*payoff[i+1,j+1]+(1-q)*payoff[i+1,j])/(1+rate_tree[i,j])
    df = pd.DataFrame(data=payoff, columns=np.arange(payoff.shape[0]))
    return df

def swaption_price(short_df,notional_principal = 1000000.0,fixed_rate = 0.045,t=5,K=0.0):
    lattice = swap_price_lattice(short_rate=short_df, notional=notional_principal, fixed_rate=fixed_rate).to_numpy()
    rate_tree= short_df.to_numpy()
    f= np.vectorize(lambda x: max(x-K,0.0))
    payoff = f(lattice)[0:t+1,0:t+1]
    for i in range(t-1,-1,-1):
        for j in range(i+1):
            payoff[i,j]=(q*payoff[i+1,j+1]+(1-q)*payoff[i+1,j])/(1+rate_tree[i,j])
    return payoff[0,0]*notional_principal

In [44]:
r00 = 0.05
n = 10
u = 1.1
d = 0.9
q = 0.5
short_df = short_rate_tree(n=n, init_rate=r00, u=u, d=d)
short_df

,0,1,2,3,4,5,6,7,8,9,10
0,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.045000,0.055000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.040500,0.049500,0.060500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.036450,0.044550,0.054450,0.066550,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.032805,0.040095,0.049005,0.059895,0.073205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.029525,0.036086,0.044105,0.053906,0.065885,0.080526,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.026572,0.032477,0.039694,0.048515,0.059296,0.072473,0.088578,0.000000,0.000000,0.000000,0.000000
7,0.023915,0.029229,0.035725,0.043663,0.053366,0.065226,0.079720,0.097436,0.000000,0.000000,0.000000
8,0.021523,0.026306,0.032152,0.039297,0.048030,0.058703,0.071748,0.087692,0.107179,0.000000,0.000000
9,0.019371,0.023676,0.028937,0.035367,0.043227,0.052833,0.064573,0.078923,0.096461,0.117897,0.000000


In [45]:
swap_df = swap_price_lattice(short_rate=short_df, notional=1000000,fixed_rate=0.045)
swap_df

,0,1,2,3,4,5,6,7,8,9,10
0,0.038136,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.002269,0.072354,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.034831,0.030090,0.102578,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-0.059296,-0.004187,0.058345,0.128223,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.075683,-0.030133,0.022286,0.081858,0.148555,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,-0.084232,-0.047710,-0.005162,0.043908,0.099823,0.162627,0.000000,0.000000,0.000000,0.000000,0.000000
6,-0.085355,-0.057132,-0.023902,0.014914,0.059826,0.111204,0.169191,0.000000,0.000000,0.000000,0.000000
7,-0.079583,-0.058807,-0.034121,-0.004968,0.029213,0.068941,0.114640,0.166559,0.000000,0.000000,0.000000
8,-0.067521,-0.053281,-0.036229,-0.015901,0.008204,0.036607,0.069818,0.108299,0.152405,0.000000,0.000000
9,-0.049810,-0.041186,-0.030791,-0.018301,-0.003345,0.014482,0.035623,0.060535,0.089672,0.123449,0.000000


### Question 1
(Pricing swaps using term structure model)

Consider an $n=10$ -period binomial model for the short-rate, $r_{i,j}$. The lattice parameters are: $r_{0,0} = 5\%$, $u=1.1$, $d=0.9$ and $q = 1-q = \frac{1}{2}$. Compute the initial value of a forward-starting swap that begins at $t=1$, with maturity $t=10$ and a fixed rate of 4.5%. The first payment then takes place at $t=2$ and the final payment takes place at $t=11$ as we are assuming, as usual, that payments take place in arrears. You should assume a swap notional of 1 million and assume that you receive floating and pay fixed.

Submission Guideline: Give your answer rounded to the nearest integer. For example, if you compute the answer to be -220,432.23, submit -220432.

In [46]:
def short_rate_tree(n,r00,u,d):
    tree = np.zeros((n+1,n+1))
    for i in range(n+1):
        for j in range(i+1):
            tree[i][j]=r00*u**j*d**(i-j)
    return tree

def forward_equation_lattice(n,r00,u,d,q=.5):
    tree = np.zeros((n+1,n+1))
    tree[0][0]=1
    rate_tree= short_rate_tree(n,r00,u,d)
    for i in range(1,n+1):
        tree[i][0]=1/(1+rate_tree[i-1][0])*(1-q)*tree[i-1][0]
        tree[i][i]=1/(1+rate_tree[i-1][i-1])*q*tree[i-1][i-1]
    for i in range(2,n+1):
        for j in range(1,i):
            tree[i,j]=(1-q)*1/(1+rate_tree[i-1,j])*tree[i-1,j]+q*1/(1+rate_tree[i-1,j-1])*tree[i-1,j-1]
    return tree

def forward_starting_swap_price(n,r00,u,d,q=.5,notional_principal = 1000000.0,fixed_rate = 0.045):
    lattice = forward_equation_lattice(n,r00,u,d,q=.5)
    # print(lattice)
    rate_tree= short_rate_tree(n,r00,u,d)
    s = 0
    for i in range(1,n+1):
        for j in range(i+1):
            s += -(fixed_rate - rate_tree[i][j])/(1+rate_tree[i][j])*lattice[i,j]
    return s*notional_principal
fp = forward_starting_swap_price(n=10, r00=0.05, u=1.1,d=0.9, q=0.5, notional_principal=1000000, fixed_rate=0.045)
print(f"Inital value of forward starting swap is {np.round(fp, 2)}")

Inital value of forward starting swap is 33374.24


### Question 2
(Pricing swaptions using term structure model)

Consider an $n=10-period$ binomial model for the short-rate,$r_{i,j}$. The lattice parameters are: $r_{0,0} = 5\%$, $u=1.1$, $d=0.9$ and $q = 1-q = \frac{1}{2}$.
Compute the initial price of a swaption that matures at time $t=5$ and has a strike of 0. The underlying swap is the same swap as described in the previous question with a notional of 1 million. To be clear, you should assume that if the swaption is exercised at $t=5$ then the owner of the swaption will receive all cash-flows from the underlying swap from times $t=6$ to $t=11$ inclusive. (The swaption strike of 0 should also not be confused with the fixed rate of 4.5% on the underlying swap.)

Submission Guideline: Give your answer rounded to the nearest integer. For example, if you compute the answer to be -220,432.23, submit -220432.

In [47]:
np.round(swaption_price(short_df=short_df, notional_principal= 1000000, fixed_rate=0.045,t=5,K=0), 2)

26311.08

### Question 3
(Calibration of term structure model)

Refer to the material on defaultable bonds and credit-default swaps (CDS).

The true price of 5 different defaultable coupon paying bonds with non-zero recovery are specified in  worksheet Calibration in the workbook Assignment5_cds.xlsx. The interest rate is $r=5\%$ per annum. Calibrate the six month hazard rates A6 to A16 by minimizing the SumError ensuring that the term structure of hazard rates are non-decreasing. You can model the non-decreasing hazard rates by adding constraints of the form A6≤A7,…,A15≤A16. Report the hazard rate at time 0 as a percentage.

Submission Guideline: Give your answer in percent rounded to two decimal places. For example, if you compute the answer to be 73.2367%, submit 73.24.

In [48]:
# The answer for this question comes from the Assignment5_CDS.xlsx and the answer is in the cell A6 which is t0 hazard rate
t0 = 1.76
print(f"The t0 Hazard Rate is {t0}%")

The t0 Hazard Rate is 1.76%


### Question 4
(Pricing Credit Default Swaps)

Refer to the material on defaultable bonds and credit-default swaps (CDS).

Modify the data on the CDS pricing worksheet in the workbook bonds_and_cds.xlsx to compute a par spread in basis points for a 5yr CDS with notional principal N=10 million assuming that the expected recovery rate R=25%, the 3-month hazard rate is a flat 1%, and the interest rate is 5% per annum.

Submission Guideline: Give your answer in basis points rounded to two decimal places (1 bps = 0.01%). For example, if you compute the answer to be 73.2367 bps, submit 73.24.


In [49]:
# The value of the Par value of 5 Yrs CDS is coming from the tab CDS Pricing using solve to make value of CDS = 0 so that spread is calculated
spread = 301.51
rate = 5
hazard_rate = 1
recovery_rate = 25
print(f"The Par value of the 5 Yr spread with interest rate {rate}% and hazard Rate {hazard_rate}% and recovery rate {recovery_rate}% is {spread} bps")

The Par value of the 5 Yr spread with interest rate 5% and hazard Rate 1% and recovery rate 25% is 301.51 bps
